# Ex2 - Getting and Knowing your Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [147]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('Spark Exercises').getOrCreate()

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

In [167]:
import pandas

#1 Using Pandas
URL_DATA = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"
pandas_df_chipotle = pandas.read_csv(URL_DATA, sep="\t")

df_chipotle = spark_context.createDataFrame(pandas_df_chipotle)
df_chipotle.printSchema()
df_chipotle.show(5)

#2 Using PySpark
FILE_PATH = "chipotle.tsv"
df_chipotle \
    = spark_context \
        .read \
        .csv(path=FILE_PATH, sep='\t', header=True, inferSchema=True)
df_chipotle.printSchema()
df_chipotle.show(5)

root
 |-- order_id: long (nullable = true)
 |-- quantity: long (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                 NaN|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                 NaN|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: 

### Step 3. Assign it to a variable called chipo.

In [149]:
chipo = df_chipotle

### Step 4. See the first 10 entries

In [150]:
# In Pandas
pandas_df_chipotle.head(10)

# In PySpark

##'head' should only be used if the resulting array is expected to be small, as all the data is loaded into the driver's memory.
df_chipotle.head(10)
df_chipotle.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

### Step 5. What is the number of observations in the dataset?

In [151]:
# In Pandas
pandas_df_chipotle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB


In [152]:
# In PySpark
df_chipotle.summary().show()

+-------+-----------------+------------------+-----------------+--------------------+----------+
|summary|         order_id|          quantity|        item_name|  choice_description|item_price|
+-------+-----------------+------------------+-----------------+--------------------+----------+
|  count|             4622|              4622|             4622|                4622|      4622|
|   mean|927.2548680225011|1.0757247944612722|             NULL|                NULL|      NULL|
| stddev|528.8907955866096|0.4101863342575333|             NULL|                NULL|      NULL|
|    min|                1|                 1|6 Pack Soft Drink|                NULL|    $1.09 |
|    25%|              477|                 1|             NULL|                NULL|      NULL|
|    50%|              926|                 1|             NULL|                NULL|      NULL|
|    75%|             1393|                 1|             NULL|                NULL|      NULL|
|    max|             1834|   

### Step 6. What is the number of columns in the dataset?

In [153]:
len(df_chipotle.columns)

5

### Step 7. Print the name of all the columns.

In [154]:
df_chipotle.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

### Step 8. How is the dataset indexed?

### Step 9. Which was the most-ordered item? 

In [155]:
from pyspark.sql import functions as F

df_chipotle_grouped \
    = df_chipotle.groupBy(df_chipotle.item_name).agg(
        F.count(df_chipotle.order_id).alias('item_order_count')
    )
df_chipotle_grouped \
    .orderBy(df_chipotle_grouped.item_order_count.desc()) \
    .select(df_chipotle_grouped.item_name) \
    .limit(1) \
    .show()

+------------+
|   item_name|
+------------+
|Chicken Bowl|
+------------+


### Step 10. For the most-ordered item, how many items were ordered?

In [156]:
df_chipotle_grouped \
    .orderBy(df_chipotle_grouped.item_order_count.desc()) \
    .select(df_chipotle_grouped.item_name, df_chipotle_grouped.item_order_count) \
    .limit(1) \
    .show()

+------------+----------------+
|   item_name|item_order_count|
+------------+----------------+
|Chicken Bowl|             726|
+------------+----------------+


### Step 11. What was the most ordered item in the choice_description column?

In [157]:
df_chipotle_grouped \
    = df_chipotle.groupBy(df_chipotle.choice_description).agg(
        F.count(df_chipotle.order_id).alias('item_order_count')
    )
df_chipotle_grouped \
    .orderBy(df_chipotle_grouped.item_order_count.desc()) \
    .select(df_chipotle_grouped.choice_description, df_chipotle_grouped.item_order_count) \
    .limit(5) \
    .show()

+--------------------+----------------+
|  choice_description|item_order_count|
+--------------------+----------------+
|                NULL|            1246|
|         [Diet Coke]|             134|
|              [Coke]|             123|
|            [Sprite]|              77|
|[Fresh Tomato Sal...|              42|
+--------------------+----------------+


### Step 12. How many items were orderd in total?

In [158]:
df_chipotle.select(F.sum(df_chipotle.quantity).alias('order_total')).show()

+-----------+
|order_total|
+-----------+
|       4972|
+-----------+


### Step 13. Turn the item price into a float

In [159]:
df_chipotle = df_chipotle.withColumn('item_price', F.expr("substring(item_price, 2)"))
df_chipotle.show(5)

df_chipotle = df_chipotle.withColumn('item_price', df_chipotle.item_price.astype('float'))
df_chipotle.printSchema()
df_chipotle.show(5)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|     2.39 |
|       1|       1|                Izze|        [Clementine]|     3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|     3.39 |
|       1|       1|Chips and Tomatil...|                NULL|     2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|    16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: float (nullable = true)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|      

#### Step 13.a. Check the item price type

In [160]:
df_chipotle.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: float (nullable = true)


#### Step 13.b. Create a lambda function and change the type of item price

#### Step 13.c. Check the item price type

### Step 14. How much was the revenue for the period in the dataset?

In [161]:
df_chipotle.select(F.sum(df_chipotle.item_price).alias('revenue')).show()

+------------------+
|           revenue|
+------------------+
|34500.159754514694|
+------------------+


### Step 15. How many orders were made in the period?

In [162]:
df_chipotle.select(F.count(df_chipotle.order_id).alias('order_count')).show()

+-----------+
|order_count|
+-----------+
|       4622|
+-----------+


### Step 16. What is the average revenue amount per order?

In [163]:
# Solution 1
df_chipotle.select(
    (F.sum(df_chipotle.item_price)/ F.count(df_chipotle.order_id)).alias('avg_revenue_per_order')
).show()

+---------------------+
|avg_revenue_per_order|
+---------------------+
|    7.464335732261942|
+---------------------+


In [164]:
# Solution 2
df_chipotle.select(
    (F.avg(df_chipotle.item_price)).alias('avg_revenue_per_order')
).show()

+---------------------+
|avg_revenue_per_order|
+---------------------+
|    7.464335732261942|
+---------------------+


In [165]:
# Solution 3
df_chipotle.select(
    (F.mean(df_chipotle.item_price)).alias('avg_revenue_per_order')
).show()

+---------------------+
|avg_revenue_per_order|
+---------------------+
|    7.464335732261942|
+---------------------+


### Step 17. How many different items are sold?

In [166]:
df_chipotle.select(
    F.count_distinct(df_chipotle.item_name).alias('distinct_item')
).show()

+-------------+
|distinct_item|
+-------------+
|           50|
+-------------+
